In [1]:
%pylab inline
import numpy as np
import sklearn.cluster as cl
from sklearn import metrics
import scipy.stats as stats

Populating the interactive namespace from numpy and matplotlib


In [2]:
params = np.loadtxt('quadratic_fit_parameters_20150505.txt')
name, W, DM = np.loadtxt("FRBs.txt", comments="#", unpack=True, usecols = [0,1,2])

In [3]:
def quad(x, a, b, c):
    return (a*x*x + b * x + c)

In [4]:
def norm_data(N,SC):
    pop_A = quad(np.linspace(2,10,N/2), params[0],params[1],params[2])
    pop_B = quad(np.linspace(1,5,N/2), params[3],params[4],params[5])
    scatter_A = pop_A + SC*np.random.randn(len(pop_A))
    scatter_B = pop_B + SC*np.random.randn(len(pop_B))
    
    total_DM = np.append(scatter_A, scatter_B)
    total_DM = np.append(total_DM, DM)
    
    total_W = np.append(np.linspace(2,10,N/2), np.linspace(1,5,N/2))
    total_W = np.append(total_W, W)
    
    total_W = np.log10(total_W)
    total_DM = np.log10(total_DM)
    
    total_DM = np.nan_to_num(total_DM)
    
    #print np.linspace(2,10,N), scatter_A
    #print np.linspace(2,10,N), scatter_B
    
    y_mean = np.mean(total_DM)
    x_mean = np.mean(total_W)
    y_diff = total_DM - y_mean
    x_diff = total_W - x_mean
    y_norm = y_diff/np.std(total_DM,ddof=1)
    x_norm = x_diff/np.std(total_W,ddof=1)
    X_norm = np.array(zip(x_norm,y_norm))

    return X_norm


In [5]:
def K_mean(X):
    kmeans_model = cl.KMeans(n_clusters=2).fit(X)
    labels = kmeans_model.labels_
    sil = metrics.silhouette_score(X,labels)
    return labels, sil

In [6]:
def Hierarchical(X):
    Ward_model = cl.Ward(n_clusters = 2).fit(X)
    labels = Ward_model.labels_
    sil = metrics.silhouette_score(X,labels)
    return labels, sil

In [7]:
def checkEqual2(iterator):
       return len(set(iterator)) <= 1
# eps=1.3,min_samples=3
def DBScan(X):
    #DBSCAN_model = cl.DBSCAN(eps=1.3).fit(X)
    tmp = cl.DBSCAN(eps=1.3*np.sqrt(10./len(X)))#, min_samples=3.*len(X)/10.)
    DBSCAN_model = tmp.fit(X)
    #print tmp.get_params("eps")
    labels = DBSCAN_model.labels_ 
    #print checkEqual2(labels)
    if checkEqual2(labels) == False:#len(np.where(labels == 0 )[0]) > 0:
        try:
            sil = metrics.silhouette_score(X,labels)
        #print sil
            return labels, sil
        except ValueError:
            #print 'labels for bad sil: ', labels
            return labels, -1.0
    else:
        return labels, -1.0

In [8]:
# choose number of bootstrap samples
def Permutation_test(data,nsims, other):
    ndata_1 = len(data)
    fakeidx1=np.floor(random.rand(ndata_1,nsims)*ndata_1)
    #print 'len(fakeidx)', np.shape(fakeidx1)
    fakeidx1 = fakeidx1.astype(int64)
    fake1=data[fakeidx1]
    #print 'fakedata', np.shape(fake1.T)
    other = np.array([other]*len(fake1.T))
    #print 'otherdata', np.shape(other)
    return zip(fake1.T, other)  

In [9]:
def p_value_K(sil, X):
    sil_K = np.zeros(len(X))
    for i in range(0,len(X)):
        permed = np.array(zip(X[i][0], X[i][1]))
        l, sil1 = K_mean(permed)
        sil_K[i] = sil1
    pval = (100. - stats.percentileofscore(sil_K,sil))/100.
    #plt.hist(sil_K)
    return pval

def p_value_H(sil, X):
    sil_K = np.zeros(len(X))
    for i in range(0,len(X)):
        permed = np.array(zip(X[i][0], X[i][1]))
        l, sil1 = Hierarchical(permed)
        sil_K[i] = sil1
    pval = (100. - stats.percentileofscore(sil_K,sil))/100.
    return pval

def p_value_DB(sil, X):
    sil_K = np.zeros(len(X))
    #print np.shape(X)
    for i in range(0,len(X)):
        permed = np.array(zip(X[i][0], X[i][1]))
        l, sil1 = DBScan(permed)
        sil_K[i] = sil1
    pval = (100. - stats.percentileofscore(sil_K,sil))/100.
    #plt.hist(sil_K)
    #plt.vlines(sil, 0, 20)
    return pval


In [10]:
def beta(p):
    b = len(np.array(np.where(p < 0.05))[0])
    return (b+0.0)/len(p)

In [11]:
def dat_to_pval_K(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = K_mean(dat)
    X2 = Permutation_test(dat[:,1], nsims, dat[:,0])
    pval = p_value_K(sil, X2)
    return pval

def dat_to_pval_H(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = Hierarchical(dat)
    X2 = Permutation_test(dat[:,1], nsims, dat[:,0])
    pval = p_value_H(sil, X2)
    return pval

def dat_to_pval_DB(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = DBScan(dat)
    #print 'sil',sil
    #print 'labels', labels
    X2 = Permutation_test(dat[:,1], nsims, dat[:,0])
    pval = p_value_DB(sil, X2)
    return pval

In [12]:
def rev_dat_to_pval_K(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = K_mean(dat)
    X2 = Permutation_test(dat[:,0], nsims, dat[:,1])
    pval = p_value_K(sil, X2)
    return pval

def rev_dat_to_pval_H(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = Hierarchical(dat)
    X2 = Permutation_test(dat[:,0], nsims, dat[:,1])
    pval = p_value_H(sil, X2)
    return pval

def rev_dat_to_pval_DB(N, SC, nsims):
    dat = norm_data(N,SC)
    labels, sil = DBScan(dat)
    #print 'sil',sil
    #print 'labels', labels
    X2 = Permutation_test(dat[:,0], nsims, dat[:,1])
    pval = p_value_DB(sil, X2)
    return pval

In [13]:
N = np.arange(0, 120, 10)

In [15]:
big_ps_K = []
bet = np.zeros(len(N))
for j in range(0, len(N)):
    ps = np.zeros(1000)
    for i in range(0, 1000):
        ps[i] = dat_to_pval_K(N[j], 100, 1000)
    bet[j] = beta(ps)
    big_ps_K.append(ps) 
    print bet
    
np.savetxt('pval_K_20150506.txt', big_ps_K)
np.savetxt('beta_K_20150506.txt', bet)

big_ps_K = np.zeros(len(N))


KeyboardInterrupt: 

In [ ]:
rev_big_ps_K = []
rev_bet = np.zeros(len(N))
for j in range(0, len(N)):
    rev_ps = np.zeros(1000)
    for i in range(0, 1000):
        rev_ps[i] = rev_dat_to_pval_K(N[j], 100, 1000)
    rev_bet[j] = beta(rev_ps)
    rev_big_ps_K.append(rev_ps) 
    print rev_bet
    
np.savetxt('rev_pval_K.txt', rev_big_ps_K)
np.savetxt('rev_beta_K.txt', rev_bet)

rev_big_ps_K = np.zeros(len(N))



In [16]:
big_ps_H = []
bet_H = np.zeros(len(N))
for j in range(0, len(N)):
    ps1 = np.zeros(1000)
    for i in range(0, 1000):
        ps1[i] = dat_to_pval_H(N[j], 100, 1000)
    bet_H[j] = beta(ps1)
    big_ps_H.append(ps1) 
    print bet_H

np.savetxt('pval_Hier_20150506.txt', big_ps_H)
np.savetxt('beta_Hier_20150506.txt', bet_H)

big_ps_H = np.zeros(len(N))


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


KeyboardInterrupt: 

In [ ]:
rev_big_ps_H = []
rev_bet_H = np.zeros(len(N))
for j in range(0, len(N)):
    rev_ps1 = np.zeros(1000)
    for i in range(0, 1000):
        rev_ps1[i] = rev_dat_to_pval_H(N[j], 100, 1000)
    rev_bet_H[j] = beta(rev_ps1)
    rev_big_ps_H.append(rev_ps1) 
    print rev_bet_H

np.savetxt('rev_pval_Hier.txt', rev_big_ps_H)
np.savetxt('rev_beta_Hier.txt', rev_bet_H)

rev_big_ps_H = np.zeros(len(N))

In [17]:
big_ps_DB = []
bet_DB = np.zeros(len(N))
for j in range(0, len(N)):
    ps2 = np.zeros(1000)
    for i in range(0, 1000):
        ps2[i] = dat_to_pval_DB(N[j], 100, 1000)
    bet_DB[j] = beta(ps2)
    big_ps_DB.append(ps2) 
    print bet_DB
    
np.savetxt('beta_DB_20150505.txt', bet_DB)
np.savetxt('pval_DB_20150505.txt', big_ps_DB)

big_ps_DB = np.zeros(len(N))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


-c:14: RuntimeWarning: invalid value encountered in log10


KeyboardInterrupt: 

In [ ]:
big_ps_DB = []
bet_DB = np.zeros(len(N))
for j in range(0, len(N)):
    ps2 = np.zeros(1000)
    for i in range(0, 1000):
        ps2[i] = dat_to_pval_DB(N[j], 100, 1000)
    bet_DB[j] = beta(ps2)
    big_ps_DB.append(ps2) 
    print bet_DB
    
np.savetxt('beta_DB1.txt', bet_DB)
np.savetxt('pval_DB1.txt', big_ps_DB)

big_ps_DB = np.zeros(len(N))

In [ ]:
rev_big_ps_DB = []
rev_bet_DB = np.zeros(len(N))
for j in range(0, len(N)):
    rev_ps2 = np.zeros(1000)
    for i in range(0, 1000):
        rev_ps2[i] = rev_dat_to_pval_DB(N[j], 100, 1000)
    rev_bet_DB[j] = beta(rev_ps2)
    rev_big_ps_DB.append(rev_ps2) 
    print rev_bet_DB
    
np.savetxt('rev_beta_DB.txt', rev_bet_DB)
np.savetxt('rev_pval_DB.txt', rev_big_ps_DB)

rev_big_ps_DB = np.zeros(len(N))

In [ ]:
rev_big_ps_DB = []
rev_bet_DB = np.zeros(len(N))
for j in range(0, len(N)):
    rev_ps2 = np.zeros(1000)
    for i in range(0, 1000):
        rev_ps2[i] = rev_dat_to_pval_DB(N[j], 100, 1000)
    rev_bet_DB[j] = beta(rev_ps2)
    rev_big_ps_DB.append(rev_ps2) 
    print rev_bet_DB
    
np.savetxt('rev_beta_DB1.txt', rev_bet_DB)
np.savetxt('rev_pval_DB1.txt', rev_big_ps_DB)

rev_big_ps_DB = np.zeros(len(N))

In [39]:

## to plot

# plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='autumn_r',s=50)

In [ ]:
#one, two = Permutation_test(dat[:,0], 20)
#print np.shape(X2)#np.array([dat[:,1]]*len(X2)))
#print len(X2)
#print np.shape(one), np.shape(two), np.shape(dat)
#print np.shape(one[:,0])
#print np.shape(np.row_stack((one,two)))
#comb = np.row_stack((one,two))
#print np.shape(comb[0])
#print np.shape(comb.T)
#print comb.T

#print 'pvalue:', (100.-stats.percentileofscore(sil_K,sil))/100.
#print sil
#sil = K_mean(zip(rrr[14][0], rrr[14][1]))

#print sil_K
#plt.hist(sil_K)
#plt.vlines(sil, 0, 300)

#print len(dat[:,0])#len(np.array(np.where(labels == 2))[0])


#dat_to_pval_K(10, 100, 1000)
#X = norm_data(10,100)
#print X
#sil = DBScan(X)
#X2 = Permutation_test(X[:,1],10, X[:,0])
#p_value_DB(sil, X2)
#print np.array(zip(X2[1][0], X2[1][1]))

#dat_to_pval_DB(100, 100, 1000)
#dat = norm_data(0,100)
#labels, sil = DBScan(dat)
#print sil , '\n'
#print labels, '\n'
#plt.figure()
#plt.scatter(dat[:, 0], dat[:, 1], c=labels, cmap='autumn_r',s=50)
#plt.figure()
#dat1 = norm_data(100,100)
#labels1, sil1 = DBScan(dat1)
#plt.scatter(dat1[:, 0], dat1[:, 1], c=labels1, cmap='autumn_r',s=50)
#print labels1, '\n', sil1